## Transformations in Political Economy - Technological Change and Populism (POL63102)
### Coding Session 7: Data Wrangling

---
This document guides you through coding session 7. Please try to follow the instructions on your own PC and feel free to ask questions if something is unclear. After this session you should be able to do the following:

- Accessing and Subsetting Data
- Merging Data
- Handling Missing Values
- Removing Duplicates
- Operations on Columns
---

Have a look at this cheat sheet for pandas methods on data wrangling here: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

As always, we first load the modules that we need for this session:

In [1]:
import pandas as pd
import numpy as np

Now we create two hypothetical data sets and save them as pandas Data Frames:
- **df1** is hypothetical individual-level data. It comes with some characteristics of individuals, including their inteded voting behaviour, gender, and occupation.
- **df2** is a data set on jobs and their  respective risk of automation.

In [2]:
data1 = {'Individual Identifier':  ['1', '2','3','4','5','5'],
        'Populist Voting Intention':  [1,0,0,np.nan,0,0],
         'Gender': ["male","","female", "male", "female", "female"],
         'Job': ['Plumber', 'Teacher','Student','Teacher','Scientist','Scientist']}

data2 = {'Job': ['Plumber', 'Teacher','Doctor','Scientist','Nurse','Bureaucrat'],
         'Risk of Automation': ['0.5', '0.4','0.3','0.2','0.1','0.8']}

df1 = pd.DataFrame (data1, columns = ['Individual Identifier','Populist Voting Intention','Gender','Job'])
df2 = pd.DataFrame (data2, columns = ['Job','Risk of Automation'])

Let's have a look at the data sets.

In [3]:
df1

,Individual Identifier,Populist Voting Intention,Gender,Job
0,1,1.0,male,Plumber
1,2,0.0,,Teacher
2,3,0.0,female,Student
3,4,NaN,male,Teacher
4,5,0.0,female,Scientist
5,5,0.0,female,Scientist


In [4]:
df2

,Job,Risk of Automation
0,Plumber,0.5
1,Teacher,0.4
2,Doctor,0.3
3,Scientist,0.2
4,Nurse,0.1
5,Bureaucrat,0.8


## Accessing and Subsetting Data

Remember that one can access columns using the slicing method with square brackets:

In [5]:
df1["Gender"] # access variable "Gender"

0      male
1          
2    female
3      male
4    female
5    female
Name: Gender, dtype: object

One can also use a condition inside the square brackets to subset data. Note the coding of logical expressions:
- == is equal
- != is not equal
-  $>$ bigger than
-  $<$ smaller than
-  $>=$ bigger or equal
-  $<=$ smaller or equal

In [6]:
df1[df1['Populist Voting Intention'] == 1] # access all observations with populist voting intention

,Individual Identifier,Populist Voting Intention,Gender,Job
0,1,1.0,male,Plumber


Note that the above cell shows data that meets the condition, but does not change the **df1** object as you can see here:

In [7]:
df1

,Individual Identifier,Populist Voting Intention,Gender,Job
0,1,1.0,male,Plumber
1,2,0.0,,Teacher
2,3,0.0,female,Student
3,4,NaN,male,Teacher
4,5,0.0,female,Scientist
5,5,0.0,female,Scientist


The *iloc()* method can be used to access rows:

In [8]:
df1.iloc[2] # access row 2 (start counting with 0)

Individual Identifier              3
Populist Voting Intention          0
Gender                        female
Job                          Student
Name: 2, dtype: object

Or to access single entries:

In [9]:
df1.iloc[2,3] # access entry in second row and third column

'Student'

**Exercise 1**: Access entries for all teachers in **df1**.

## Merging Data Sets

To investigate the relationship between risk of automation and populist voting intention, we need to merge the two data sets. As you can see, the key variable to merge on is *job*. We first have to make sure that this variable is of the same type in both data sets (namely, string class):

In [10]:
df1['Job'] = df1['Job'].astype(str)
df2['Job'] = df2['Job'].astype(str)

Now we can use the *merge()* function from **pandas**.

In [11]:
df3 = pd.merge(df1,df2,how='left',on='Job')
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation
0,1,1.0,male,Plumber,0.5
1,2,0.0,,Teacher,0.4
2,3,0.0,female,Student,NaN
3,4,NaN,male,Teacher,0.4
4,5,0.0,female,Scientist,0.2
5,5,0.0,female,Scientist,0.2


**Exercise 2:** What has happened here? What is the difference betweem the 'left' and 'right', 'inner' and 'outer' merges? You might want to consult the second page of cheat sheet:https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

## Dropping Missing Data
You will have noticed that some value entries are **NaN**. This is python's way to tell you that this is a *missing value*. Missing values can cause problems in analysis, so ideally they are addressed before running regressions. 

There might be various reasons why values are missing. Under the assumption of "missing at random", we can simply drop the missing values and proceed with our analysis. Good research discusses why certain values might be missing and in which direction this might bias coefficients. However, regression results with different specifications should be reported using identical samples. The reason is that differences in coefficients might be due to either differences in specifications or samples, so we need to take the same sample to be able to make statements about how coefficients change with different specifications.

Another way to deal with missing data is *imputation*. However, imputation also comes with assumptions and problems, so let's stick with dropping missing values for this course.

Of course, it is important to know how many values in the data are missing. Let's ask Python to show us how many missings there are for each variable:

In [12]:
df1.isna().sum()

Individual Identifier        0
Populist Voting Intention    1
Gender                       0
Job                          0
dtype: int64

Let's use *dropna()* to drop missing values. Note that we need to overwrite our previous **df3** object to store the data set without missing values in memory.

In [13]:
df3 = df3.dropna()
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation
0,1,1.0,male,Plumber,0.5
1,2,0.0,,Teacher,0.4
4,5,0.0,female,Scientist,0.2
5,5,0.0,female,Scientist,0.2


You probably noticed that one missing value (gender of the teacher) was not explicitly declared as **NaN**, but is just an empty string. We need to deal with this, too.  Let's convert empty strings to proper **NaN** values and get rid of observations with missing values again.

In [14]:
df3[:].replace("", np.nan, inplace=True)

In [15]:
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation
0,1,1.0,male,Plumber,0.5
1,2,0.0,NaN,Teacher,0.4
4,5,0.0,female,Scientist,0.2
5,5,0.0,female,Scientist,0.2


In [16]:
df3 = df3.dropna()
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation
0,1,1.0,male,Plumber,0.5
4,5,0.0,female,Scientist,0.2
5,5,0.0,female,Scientist,0.2


## Removing Duplicates
You might have noticed that the female scientist with the same identifier appears twice in our data. To remove duplicates, we can use the function *drop_duplicates()*. We specify that duplicates of the individual identifier column should be removed:

In [17]:
df3 = df3.drop_duplicates(subset=['Individual Identifier'])
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation
0,1,1.0,male,Plumber,0.5
4,5,0.0,female,Scientist,0.2


## Operations on Columns
You might want to change the scale of one of your variables. For example, let's create a new column that consists of the Risk of Automation multiplied by 10.

Note that the following produces an error:

In [18]:
df3["Risk Automation (0-10)"] = 10 * df3["Risk of Automation"]
df3

C:\Users\felix\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation,Risk Automation (0-10)
0,1,1.0,male,Plumber,0.5,0.50.50.50.50.50.50.50.50.50.5
4,5,0.0,female,Scientist,0.2,0.20.20.20.20.20.20.20.20.20.2


The reason is that entries in the column Risk of Automation are not a numeric, but string. Let's convert the string entries to numeric format using pandas *to_numeric()* function (you can ignore the error warning here):

In [19]:
df3["Risk of Automation (numeric)"] = pd.to_numeric(df3["Risk of Automation"], errors='coerce') # there are missing values, which are now NaN

C:\Users\felix\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation,Risk Automation (0-10),Risk of Automation (numeric)
0,1,1.0,male,Plumber,0.5,0.50.50.50.50.50.50.50.50.50.5,0.5
4,5,0.0,female,Scientist,0.2,0.20.20.20.20.20.20.20.20.20.2,0.2


Then, let's delete the columns for risk of automation with the string entries using the *drop()* function:

In [21]:
df3 = df3.drop(columns = ["Risk Automation (0-10)", "Risk of Automation"])
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation (numeric)
0,1,1.0,male,Plumber,0.5
4,5,0.0,female,Scientist,0.2


Now we can create our new variable "Risk Automation (0-10)":

In [22]:
df3["Risk Automation (0-10)"] = 10 * df3["Risk of Automation (numeric)"]
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation (numeric),Risk Automation (0-10)
0,1,1.0,male,Plumber,0.5,5.0
4,5,0.0,female,Scientist,0.2,2.0


We can also change a variable name using the *rename()* function:

In [23]:
df3 = df3.rename(columns = {"Risk Automation (0-10)" : 'Automation Risk'})
df3

,Individual Identifier,Populist Voting Intention,Gender,Job,Risk of Automation (numeric),Automation Risk
0,1,1.0,male,Plumber,0.5,5.0
4,5,0.0,female,Scientist,0.2,2.0


**Exercise 3:** Rename the variable *Individual Identifier* to *ID". Create a dummy variable for gender that takes the value 0 if male and 1 if female.

**Exercise 4:** Have a look at the *groupby()* function. What does this function do?

In [25]:
help(df1.groupby)

Help on method groupby in module pandas.core.generic:

groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False, **kwargs) method of pandas.core.frame.DataFrame instance
    Group DataFrame or Series using a mapper or by a Series of columns.
    
    A groupby operation involves some combination of splitting the
    object, applying a function, and combining the results. This can be
    used to group large amounts of data and compute operations on these
    groups.
    
    Parameters
    ----------
    by : mapping, function, label, or list of labels
        Used to determine the groups for the groupby.
        If ``by`` is a function, it's called on each value of the object's
        index. If a dict or Series is passed, the Series or dict VALUES
        will be used to determine the groups (the Series' values are first
        aligned; see ``.align()`` method). If an ndarray is passed, the
        values are used as-is determine t

---
**Congratulations! This is the end of coding session 7.**

Further Resources on Pandas and Data Wrangling: 
- https://pandas.pydata.org/docs/user_guide/10min.html
- https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_stata.html
- https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf
